Try four different things:

1. Does `Mish` affect this dataset
2. Self Attention?
3. Adam + One Cycle 
4. Ranger + flat_cos

Fit for 100 epoch batches

In [0]:
import os
!pip install -q feather-format kornia pyarrow wandb nbdev fastprogress  --upgrade 
!pip install torch==1.3.1
!pip install torchvision==0.4.2
os._exit(00)

In [0]:
!pip install git+https://github.com/fastai/fastai2.git
!pip install git+https://github.com/fastai/fastcore.git

In [0]:
from fastai2.tabular.all import *

In [0]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
0,4,9,2,1,2,2,4,7,2,8,0
1,1,4,3,6,1,12,3,11,2,7,0
2,1,11,4,1,3,7,4,11,2,1,2
3,2,9,2,4,3,6,1,9,4,9,3
4,1,8,2,4,2,11,2,2,2,1,0


Now we'll represent the card number as a categorical too

In [0]:
for i in range(1,6):
    df['cC'+str(i)]=df['C'+str(i)]

In [0]:
cat_vars = ['S1', 'S2', 'S3', 'S4', 'S5','cC1', 'cC2', 'cC3', 'cC4', 'cC5']
cont_vars = ['C1', 'C2', 'C3', 'C4', 'C5']
dep_var = ['hand']

In [0]:
procs = [Categorify, Normalize]
splits = RandomSplitter()(range_of(df))

In [0]:
to = TabularPandas(df, procs, cat_names=cat_vars, cont_names=cont_vars, y_names=dep_var, splits=splits)

In [8]:
to.show()

,S1,S2,S3,S4,S5,cC1,cC2,cC3,cC4,cC5,C1,C2,C3,C4,C5,hand
17218,3,3,1,3,1,5,3,3,4,4,5.0,3.0,3.0,4.0,4.0,2
23181,4,4,2,2,2,10,13,6,2,12,10.0,13.0,6.0,2.0,12.0,0
11213,3,1,1,2,3,13,7,8,1,1,13.0,7.0,8.0,1.0,1.0,1
13364,1,4,3,3,1,9,5,13,1,10,9.0,5.0,13.0,1.0,10.0,0
13937,3,2,4,3,2,10,7,2,12,2,10.0,7.0,2.0,12.0,2.0,1
17760,4,4,1,2,1,9,10,1,4,10,9.0,10.0,1.0,4.0,10.0,1
5463,2,4,3,4,2,9,6,7,13,13,9.0,6.0,7.0,13.0,13.0,1
18348,2,4,4,4,4,10,10,7,8,1,10.0,10.0,7.0,8.0,1.0,1
4866,4,3,1,2,3,5,3,5,12,8,5.0,3.0,5.0,12.0,8.0,1
20850,2,3,1,3,4,11,12,5,4,9,11.0,12.0,5.0,4.0,9.0,0


In [20]:
len(to)

25010

In [0]:
dls = to.dataloaders(bs=2048)

# Baseline

In [0]:
class myLabelSmoothingCrossEntropyFlat(BaseLoss):
    "Same as `nn.CrossEntropyLoss`, but flattens input and target."
    y_int = True
    def __init__(self, *args, axis=-1, **kwargs): super().__init__(LabelSmoothingCrossEntropy, *args, axis=axis, **kwargs)
    def decodes(self, x):    return x.argmax(dim=self.axis)
    def activation(self, x): return F.softmax(x, dim=self.axis)

In [0]:
class TrackHighestEpoch(TrackerCallback):
    "A `TrackerCallback` that terminates training when monitored quantity stops improving."
    def __init__(self, monitor='accuracy', comp=None, min_delta=0., patience=1):
        super().__init__(monitor=monitor, comp=comp, min_delta=min_delta)
        self.patience = patience

    def begin_fit(self): self.highest_epoch = 0; super().begin_fit()
    def after_epoch(self):
        "Compare the value monitored to its best score and maybe stop training."
        super().after_epoch()
        if self.new_best: 
          self.highest_epoch = self.epoch
    def after_fit(self):
        "Print highest value"
        print(f'Highest accuracy of: {self.best} at epoch {self.highest_epoch+1}')

## Run 1

In [108]:
learn = tabular_learner(dls, layers=[100,50,50],
                        ps=[0.01,0.01,0.02], metrics=accuracy,
                        loss_func=myLabelSmoothingCrossEntropyFlat(), n_out=10,
                        cbs=TrackHighestEpoch)
with learn.no_bar():
  learn.fit_one_cycle(400, 1e-1)

(#5) [0,1.8988388776779175,2.0034022331237793,0.44682127237319946,'00:00']
(#5) [1,1.6181501150131226,1.3519799709320068,0.5001999139785767,'00:00']
(#5) [2,1.504436731338501,1.327201008796692,0.5179927945137024,'00:00']
(#5) [3,1.4352670907974243,1.2917509078979492,0.5171931385993958,'00:00']
(#5) [4,1.3906110525131226,1.2875611782073975,0.5209916234016418,'00:00']
(#5) [5,1.3558152914047241,1.2735843658447266,0.5453818440437317,'00:00']
(#5) [6,1.327859878540039,1.2561819553375244,0.5711715221405029,'00:00']
(#5) [7,1.302958607673645,1.2359035015106201,0.5897641181945801,'00:00']
(#5) [8,1.2801798582077026,1.2195144891738892,0.6043582558631897,'00:00']
(#5) [9,1.2572294473648071,1.2002384662628174,0.6247500777244568,'00:00']
(#5) [10,1.2347543239593506,1.1807585954666138,0.6387444734573364,'00:00']
(#5) [11,1.2118557691574097,1.1614450216293335,0.658736526966095,'00:00']
(#5) [12,1.1889569759368896,1.1361898183822632,0.6761295199394226,'00:00']
(#5) [13,1.1659647226333618,1.128595948

## Run 2

In [109]:
learn = tabular_learner(dls, layers=[100,50,50],
                        ps=[0.01,0.01,0.02], metrics=accuracy,
                        loss_func=myLabelSmoothingCrossEntropyFlat(), n_out=10,
                        cbs=TrackHighestEpoch)
with learn.no_bar():
  learn.fit_one_cycle(400, 1e-1)

(#5) [0,1.8379777669906616,1.9357826709747314,0.49500200152397156,'00:00']
(#5) [1,1.576326847076416,1.3467679023742676,0.49500200152397156,'00:00']
(#5) [2,1.4765552282333374,1.2920480966567993,0.4968012869358063,'00:00']
(#5) [3,1.415789008140564,1.2997297048568726,0.5057976841926575,'00:00']
(#5) [4,1.3760191202163696,1.2883087396621704,0.5195921659469604,'00:00']
(#5) [5,1.3465536832809448,1.2797282934188843,0.5377848744392395,'00:00']
(#5) [6,1.3234179019927979,1.2694367170333862,0.545781672000885,'00:00']
(#5) [7,1.3030749559402466,1.2590352296829224,0.558976411819458,'00:00']
(#5) [8,1.2843879461288452,1.2440656423568726,0.5703718662261963,'00:00']
(#5) [9,1.265557885169983,1.2323834896087646,0.588164746761322,'00:00']
(#5) [10,1.2474874258041382,1.207640528678894,0.6109555959701538,'00:00']
(#5) [11,1.2271007299423218,1.183942198753357,0.6329468488693237,'00:00']
(#5) [12,1.2050892114639282,1.1692123413085938,0.6437425017356873,'00:00']
(#5) [13,1.1826720237731934,1.17051017284

## Run 3

In [110]:
learn = tabular_learner(dls, layers=[100,50,50],
                        ps=[0.01,0.01,0.02], metrics=accuracy,
                        loss_func=myLabelSmoothingCrossEntropyFlat(), n_out=10,
                        cbs=TrackHighestEpoch)
with learn.no_bar():
  learn.fit_one_cycle(400, 1e-1)

(#5) [0,2.0293729305267334,2.0614888668060303,0.42463013529777527,'00:00']
(#5) [1,1.6987190246582031,1.3678958415985107,0.4296281635761261,'00:00']
(#5) [2,1.5585050582885742,1.3144861459732056,0.5019991993904114,'00:00']
(#5) [3,1.4771995544433594,1.3003623485565186,0.514394223690033,'00:00']
(#5) [4,1.4260493516921997,1.2955440282821655,0.5125949382781982,'00:00']
(#5) [5,1.3896437883377075,1.2868778705596924,0.5139943957328796,'00:00']
(#5) [6,1.363073468208313,1.2853848934173584,0.5255897641181946,'00:00']
(#5) [7,1.3416186571121216,1.280599594116211,0.531987190246582,'00:00']
(#5) [8,1.3240435123443604,1.2781611680984497,0.5329868197441101,'00:00']
(#5) [9,1.3087990283966064,1.2757419347763062,0.5391843318939209,'00:00']
(#5) [10,1.2953828573226929,1.2710528373718262,0.5387845039367676,'00:00']
(#5) [11,1.2829558849334717,1.263350009918213,0.5507797002792358,'00:00']
(#5) [12,1.2711344957351685,1.248146414756775,0.5709716081619263,'00:00']
(#5) [13,1.2579454183578491,1.2261199951

## Run 4

In [111]:
learn = tabular_learner(dls, layers=[100,50,50],
                        ps=[0.01,0.01,0.02], metrics=accuracy,
                        loss_func=myLabelSmoothingCrossEntropyFlat(), n_out=10,
                        cbs=TrackHighestEpoch)
with learn.no_bar():
  learn.fit_one_cycle(400, 1e-1)

(#5) [0,2.0003058910369873,2.0761032104492188,0.42463013529777527,'00:00']
(#5) [1,1.67828369140625,1.42345130443573,0.4906037449836731,'00:00']
(#5) [2,1.5407624244689941,1.3101568222045898,0.49840062856674194,'00:00']
(#5) [3,1.4638121128082275,1.2966821193695068,0.49740102887153625,'00:00']
(#5) [4,1.4130326509475708,1.2953085899353027,0.5041983127593994,'00:00']
(#5) [5,1.3762407302856445,1.2870265245437622,0.5175929665565491,'00:00']
(#5) [6,1.348793625831604,1.273974895477295,0.5483806729316711,'00:00']
(#5) [7,1.3258235454559326,1.2650690078735352,0.5631747245788574,'00:00']
(#5) [8,1.305737018585205,1.2540934085845947,0.5685725808143616,'00:00']
(#5) [9,1.2874256372451782,1.2418713569641113,0.5783686637878418,'00:00']
(#5) [10,1.269816279411316,1.2248858213424683,0.588764488697052,'00:00']
(#5) [11,1.2515718936920166,1.2058950662612915,0.6143542528152466,'00:00']
(#5) [12,1.23350989818573,1.1945611238479614,0.6259496212005615,'00:00']
(#5) [13,1.2134891748428345,1.1741676330566

## Run 5

In [112]:
learn = tabular_learner(dls, layers=[100,50,50],
                        ps=[0.01,0.01,0.02], metrics=accuracy,
                        loss_func=myLabelSmoothingCrossEntropyFlat(), n_out=10,
                        cbs=TrackHighestEpoch)
with learn.no_bar():
  learn.fit_one_cycle(400, 1e-1)

(#5) [0,1.856536865234375,1.9822254180908203,0.49500200152397156,'00:00']
(#5) [1,1.5840586423873901,1.3882856369018555,0.49500200152397156,'00:00']
(#5) [2,1.4862524271011353,1.298932671546936,0.49500200152397156,'00:00']
(#5) [3,1.4239410161972046,1.3021901845932007,0.5035985708236694,'00:00']
(#5) [4,1.3853504657745361,1.3002160787582397,0.5053978562355042,'00:00']
(#5) [5,1.3564832210540771,1.2835593223571777,0.526989221572876,'00:00']
(#5) [6,1.3348164558410645,1.2770748138427734,0.5327869057655334,'00:00']
(#5) [7,1.3162322044372559,1.2688472270965576,0.5463814735412598,'00:00']
(#5) [8,1.2997393608093262,1.2645835876464844,0.55677729845047,'00:00']
(#5) [9,1.2839065790176392,1.2528749704360962,0.5687724947929382,'00:00']
(#5) [10,1.2694367170333862,1.2479283809661865,0.5867652893066406,'00:00']
(#5) [11,1.254752516746521,1.2275829315185547,0.5963614583015442,'00:00']
(#5) [12,1.2393661737442017,1.2126461267471313,0.6097561120986938,'00:00']
(#5) [13,1.2225242853164673,1.20127236

In [0]:
avgs = np.array([99.48,99.46,99.44,99.4,99.44])
epchs = np.array([331,323,267,298,278])

In [131]:
print(f"Average Accuracy: {avgs.mean()}\nAccuracy Stdv: {avgs.std()}\n\
Average Total Epochs: {epchs.mean()}\nEpcohs Stdv: {epchs.std()}")

Average Accuracy: 99.44399999999999
Accuracy Stdv: 0.026532998322841776
Average Total Epochs: 299.4
Epcohs Stdv: 24.759644585494357
